In [1]:
import requests
import json
from tqdm import tqdm
import pandas as pd

# 보 관측소 api
- 보 관측소 제원 요청

In [ ]:
url = 'http://api.hrfco.go.kr/A5F02E26-FBD3-47F7-A191-280B5944845D/bo/info.json'
resp = requests.get(url)
json_row = resp.json()['content']
json_row

In [29]:
def bo_station():
    url = 'http://api.hrfco.go.kr/A5F02E26-FBD3-47F7-A191-280B5944845D/bo/info.json'
    
    resp = requests.get(url)
    
    json_row = resp.json()['content']
    
    code = [] #보 관측소 코드
    name = [] #보 관측소 명칭
    agency = [] #관할 기관
    addr = [] #주소
    etcaddr = [] #나머지주소
    lat = [] #위도
    lon = [] #경도
    pfh = [] #계획홍수위
    spcwl = [] #관리수위
    
    for i in range(len(json_row)):
        row = json_row[i]
        
        if row == None:
            pass
        else:
            code.append(row['boobscd'])
            name.append(row['obsnm'])
            agency.append(row['agcnm']) 
            addr.append(row['addr'])
            etcaddr.append(row['etcaddr'])
            lat.append(row['lat'])
            lon.append(row['lon'])
            pfh.append(row['pfh'])
            spcwl.append(row['spcwl'])
        
    df = pd.DataFrame({"station_code": code, "agency_name": agency, "station_name":name, "address":addr, "etc_address":etcaddr,
                       "latitude": lat, "longitude": lon, "pfh": pfh, "spcwl": spcwl})
    
    return df

In [33]:
bo_station_df = bo_station()
bo_staion_code = [ bo_station_df['station_code'][i] for i in range(len(bo_station_df)) ]

# 보 api
- 보 자료 요청

In [ ]:
url = 'http://api.hrfco.go.kr/A5F02E26-FBD3-47F7-A191-280B5944845D/bo/list/1H.json'
resp = requests.get(url)
json_row = resp.json()['content']
json_row

In [46]:
bo_station_df = bo_station()
bo_station_code = [ bo_station_df['station_code'][i] for i in range(len(bo_station_df)) ]

def bo_api():
    global bo_station_code
    url = 'http://api.hrfco.go.kr/A5F02E26-FBD3-47F7-A191-280B5944845D/bo/list/1H.json'
    
    resp = requests.get(url)
    
    json_row = resp.json()['content']
    
    #station_code = [ json_row[i]['boobscd'] for i in range(len(json_row)) ]
    
    url_list = ['http://api.hrfco.go.kr/A5F02E26-FBD3-47F7-A191-280B5944845D/bo/list/1H/' + i for i in bo_station_code]
    
    date = [ '/20' + str(i) + '010100/20' + str(i) + '123123.json' for i in range(10,21) ]
    
    code = [] #보 관측소 코드
    YMDHM = [] #년월일시분
    SWL = [] #보 상류수위
    OWL = [] #하류수위
    INF = [] #유입량
    SFW = [] #저수량
    ECPC = [] #공용량
    TOTOTF = [] #총 방유량
    
    for i in tqdm(url_list):
        for j in date:
            url = i + j
            resp = requests.get(url)
            json_row = resp.json()['content']
            
            for k in range(len(json_row)):
                row = json_row[k]
                code.append(row['boobscd'])
                YMDHM.append(row['ymdhm'])
                SWL.append(row['swl'])
                OWL.append(row['owl'])
                INF.append(row['inf'])
                SFW.append(row['sfw'])
                ECPC.append(row['ecpc'])
                TOTOTF.append(row['tototf'])
                
    df = pd.DataFrame({'station_code':code, 'obs_date':YMDHM, 'swl':SWL, 'owl':OWL, 'inf':INF,
                      'sfw':SFW, 'ecpc':ECPC, 'tototf':TOTOTF})
    
    return df

In [47]:
bo_df = bo_api()

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [22:27<00:00, 79.25s/it]


In [48]:
bo_df.to_csv("보_api.csv")